If you're not running in Saturn Cloud, you need to install these libraries:

Make sure you use the latest versions

```
pip install -U transformers accelerate bitsandbytes
```

In [1]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-07 16:20:52--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-07 16:20:52 (34.0 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [3]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [4]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [9]:
llm('write this is a test')

' This seems to be an instruction for generating or creating content that tests something, likely in the context of software development. Given it\'s incomplete and vague without any specific target language mentioned, I will create one possible interpretation wherein we are testing whether my capabilities as an AI extend across multiple languages:\n\n```json\n{\n  "test": {\n    "description": "Multilingual Text Translation Test",\n    "input_phrase": {\n      "source_language": "English",\n      "text": "Write this is a test"\n    },\n    "expected_outcome": {\n      "various_languages": [\n        {"French": "Écrire ceci est un test"},\n        {"Spanish": "Escriba esto es una prueba"},\n        {"German": "Schreibe dies ist ein Test"}\n      ]\n    }\n  }\n}\n```'

In [10]:
print(_)

 This seems to be an instruction for generating or creating content that tests something, likely in the context of software development. Given it's incomplete and vague without any specific target language mentioned, I will create one possible interpretation wherein we are testing whether my capabilities as an AI extend across multiple languages:

```json
{
  "test": {
    "description": "Multilingual Text Translation Test",
    "input_phrase": {
      "source_language": "English",
      "text": "Write this is a test"
    },
    "expected_outcome": {
      "various_languages": [
        {"French": "Écrire ceci est un test"},
        {"Spanish": "Escriba esto es una prueba"},
        {"German": "Schreibe dies ist ein Test"}
      ]
    }
  }
}
```
